In [1]:
#!pip install node
#!pip install npm
#!pip install sodapy
from sodapy import Socrata
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import os
import datetime

from config import austin_key
from config import austin_app

In [2]:
#need to install extensions in vscode for instant markdown(by david bankier)
#allows you to open a readme inside vscode that looks how the formatting should when you load it to git hub 
#allows you to real time edit the readme and you can see how the readme formatting changes in git hub 
#concept of bullets in markdown is asterisk (will be formatted as bullets in the git file)
#*italic*
#**bold**
#***bold italic***
#--- will create a separating line across the file
#insert pictures with [caption](link to pic in file structure)  ex. ()
# number of ## in front of the text will change the size (more # means bigger font)
# tab indent below a bullet with another * will format a hollow sub bulllet
# using the `` around a phrase will format it as a code snippet (gray background and typewriter font)
# ||| will be a column separator to make tables from scratch
#|header 1|header 2
#------|-----
#|h1 data|h2 data
# : tells the table where to align the words (-----: rt align :----- left align)
#check out markdown cheat sheet on wikipedia

In [15]:
#not going to use the FBI data because it is submitted annually by the individual agencies so the data in the reports is compiled on an annual basis 
#we need data that is by day so we can see trends across the monthly 
#api_key=fbi_api_key
#state = "state"
#url = "https://api.usa.gov/crime/fbi/sapi/"
#query_url = f"https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/TX1010000/homicide/2017/2019?API_KEY=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv"
#query_url
#client = Socrata("data.austintexas.gov", None)
#results = client.get("fdj4-gpfu", limit=2000)
#results_df = pd.DataFrame.from_records(results)

#results_df.head()

app_token= austin_app

url = "https://data.austintexas.gov/resource/fdj4-gpfu.json?"
app_token= austin_app
crime= "DEL CONTROLLED SUB/NARCOTIC"

#query_url = f"{url}$$app_token={app_token}&crime_type={crime}"
query_url = f"{url}$$app_token={app_token}"
query_url

'https://data.austintexas.gov/resource/fdj4-gpfu.json?$$app_token=2rp1GusN9cI4uGmGNPn2jlU7I'

In [24]:
response = requests.get(query_url).json()
#response[1]['crime_type']
print(json.dumps(response, indent=4))

In [25]:
crime_df = pd.DataFrame(response)
crime_df.head()

incident_report_number                   crime_type ucr_code  \
0             2021581342  DEL CONTROLLED SUB/NARCOTIC     1804   
1            20215007524                        THEFT      600   
2            20215007489          ASSAULT WITH INJURY      900   
3            20215007512        BURGLARY OF RESIDENCE      500   
4            20215007516                        THEFT      600   

  family_violence            occ_date_time                 occ_date occ_time  \
0               N  2021-02-27T20:08:00.000  2021-02-27T00:00:00.000     2008   
1               N  2021-02-27T18:30:00.000  2021-02-27T00:00:00.000     1830   
2               N  2021-02-27T13:05:00.000  2021-02-27T00:00:00.000     1305   
3               N  2021-02-27T13:00:00.000  2021-02-27T00:00:00.000     1300   
4               N  2021-02-27T13:00:00.000  2021-02-27T00:00:00.000     1300   

             rep_date_time                 rep_date rep_time  ...  \
0  2021-02-27T20:08:00.000  2021-02-27T00:00:00.000     2008  ...   
1  2021-02-27T23:09:00.000  2021-02-27T00:00:00.000     2309  ...   
2  2021-02-27T13:05:00.000  2021-02-27T00:00:00.000     1305  ...   
3  2021-02-27T19:31:00.000  2021-02-27T00:00:00.000     1931  ...   
4  2021-02-27T20:26:00.000  2021-02-27T00:00:00.000     2026  ...   

      longitude                                           location  \
0  -97.77011645  {'latitude': '30.19024097', 'longitude': '-97....   
1  -97.74208894  {'latitude': '30.26871572', 'longitude': '-97....   
2  -97.69116892  {'latitude': '30.3391525', 'longitude': '-97.6...   
3  -97.76315586  {'latitude': '30.48249752', 'longitude': '-97....   
4  -97.79798512  {'latitude': '30.47104169', 'longitude': '-97....   

  :@computed_region_a3it_2a2z :@computed_region_8spj_utxs  \
0                        3258                           2   
1                        2856                           9   
2                        3642                           4   
3                        2862                           6   
4                        2862                           6   

  :@computed_region_q9nd_rr82 ucr_category category_description  \
0                           4          NaN                  NaN   
1                          10          23H                Theft   
2                           9          NaN                  NaN   
3                           6          220             Burglary   
4                           6          23H                Theft   

  :@computed_region_qwte_z96m clearance_status clearance_date  
0                         NaN              NaN            NaN  
1                        2749              NaN            NaN  
2                        2233              NaN            NaN  
3                        3476              NaN            NaN  
4                         NaN              NaN            NaN  

[5 rows x 30 columns]

In [23]:
#think of some crimes you think are more or less likely to be committed under a full moon/ at night etc
#adding databases together vertically will be concatenate; filter databases down to crime type and occurence date 
#pd.concat[]
all_crimes = [response[i]['crime_type'] for i in range (len(response))]
set(all_crimes)

{'ABUSE OF 911',
 'AGG ASLT STRANGLE/SUFFOCATE',
 'AGG ASLT W/MOTOR VEH FAM/DAT V',
 'AGG ASSAULT',
 'AGG ASSAULT FAM/DATE VIOLENCE',
 'AGG RAPE',
 'AGG ROBBERY BY ASSAULT',
 'AGG ROBBERY/DEADLY WEAPON',
 'AGG SEXUAL ASSAULT CHILD/OBJEC',
 'AGG SEXUAL ASSAULT W OBJECT',
 'AIRPORT - FEDERAL VIOL',
 'ARSON',
 'ASSAULT  CONTACT-SEXUAL NATURE',
 'ASSAULT BY CONTACT',
 'ASSAULT BY CONTACT FAM/DATING',
 'ASSAULT BY THREAT',
 'ASSAULT OF PREGNANT WM-FAM/DAT',
 'ASSAULT ON PEACE OFFICER',
 'ASSAULT W/INJURY-FAM/DATE VIOL',
 'ASSAULT WITH INJURY',
 'AUTO THEFT',
 'BREACH OF COMPUTER SECURITY',
 'BURG OF RES - FAM/DATING ASLT',
 'BURGLARY NON RESIDENCE',
 'BURGLARY OF RESIDENCE',
 'BURGLARY OF SHED/DETACHED GARAGE/STORAGE UNIT',
 'BURGLARY OF VEH-NO SUSPECT/FU',
 'BURGLARY OF VEHICLE',
 'CHILD CUSTODY INTERFERE',
 'CRASH/FAIL STOP AND RENDER AID',
 'CRED CARD ABUSE - OTHER',
 'CRIMINAL MISCHIEF',
 'CRIMINAL MISCHIEF-NO SUSPECT',
 'CRIMINAL TRESPASS',
 'CRIMINAL TRESPASS/HOTEL',
 'CRIMINAL TRESPA